In [18]:
import os
import numpy as np
import matplotlib.pyplot as plt
import struct
from scipy.signal import find_peaks
from ipywidgets import Text, fixed, VBox, Output

def view_chn_spectrum(filepath, a=0.172, b=-5.92, prominence=500, width=600, show_line=True):
#     def read_ortec_chn(filepath):
#         with open(filepath, 'rb') as f:
#             data = f.read()
#         num_channels = struct.unpack('<H', data[20:22])[0]
#         spectrum_data = struct.unpack('<' + 'H' * num_channels, data[32:32 + 2 * num_channels])
#         return np.array(spectrum_data)

    def read_ortec_chn(filepath):
        with open(filepath, 'rb') as f:
            data = f.read()
        num_channels = struct.unpack('<H', data[20:22])[0]
        spectrum_data = struct.unpack('<' + 'H' * num_channels, data[32:32 + 2 * num_channels])
        full_counts = np.array(spectrum_data)

        # Use only the half that is non-zero
        if np.all(full_counts[::2] == 0):
            counts = full_counts[1::2]  # use odd bins
            offset = 1
        elif np.all(full_counts[1::2] == 0):
            counts = full_counts[::2]  # use even bins
            offset = 0
        else:
            counts = full_counts
            offset = 0

        return counts, offset

    def channel_to_energy(channels, a, b):
        return a * channels + b

    def plot_spectrum(counts, a, b, x_value, show_line):
#         channels = np.arange(len(counts))
#         energy = channel_to_energy(channels, a, b)
        
        channels = np.arange(len(counts)) * 2 + offset
        energy = channel_to_energy(channels, a, b)

        # Snap to nearest bin
        index = np.argmin(np.abs(energy - x_value))
        snapped_energy = energy[index]
        value_at_x = counts[index]

        fig, ax = plt.subplots(figsize=(12, 6))
        ax.plot(energy, counts, label='Counts')
        ax.set_yscale('log')

        # Conditionally show red line and text
        if show_line:
            ax.axvline(x=snapped_energy, color='r', linestyle='--')
            ax.scatter(snapped_energy, value_at_x, color='r')
            ax.text(snapped_energy + 0.2, value_at_x + 0.1,
                    f'Value: {value_at_x:.2f}\nEnergy: {snapped_energy:.2f} keV',
                    color='r', fontsize=9)

        # Add labels, title, etc.
        ax.set_title(f'Spectrum from {filepath}')
        ax.set_xlabel('Energy (keV)')
        ax.set_ylabel('log(Counts)')
        ax.grid(True)
        ax.legend()
        plt.tight_layout()
        plt.show()

    # Load data
#     counts = read_ortec_chn(filepath)
    counts, offset = read_ortec_chn(filepath)
    channels = np.arange(len(counts))
    energy = channel_to_energy(channels, a, b)
    energy_min, energy_max = energy[0], energy[-1]

    # Text box widget
    energy_text = Text(
        value=str(0),
        description='Energy:',
        continuous_update=False
    )

    out = Output()

    def update_plot(change):
        with out:
            out.clear_output()
            try:
                val = float(change['new'])
                if energy_min <= val <= energy_max:
                    plot_spectrum(counts, a, b, val, show_line)
            except ValueError:
                pass  # Invalid input — ignore

    energy_text.observe(update_plot, names='value')

    # Show initial plot
    with out:
        plot_spectrum(counts, a, b, float(energy_text.value), show_line)

    return VBox([energy_text, out])

In [19]:
folder = "C:\\Users\\livia\\Downloads\\College\\PHYS 4410\\N-0\\Data\\4-30-25"
files = [os.path.join(folder, file_path) for file_path in os.listdir(folder)]

for f in files:
    display(view_chn_spectrum(f, a=0.2878, b=-9.14, show_line=True));

In [16]:
import struct
import numpy as np

def read_ortec_chn(filepath):
    with open(filepath, 'rb') as f:
        data = f.read()
    num_channels = struct.unpack('<H', data[20:22])[0]
    spectrum_data = struct.unpack('<' + 'H' * num_channels, data[32:32 + 2 * num_channels])
    return np.array(spectrum_data)

def channel_to_energy(channels, a, b):
    return a * channels + b

def print_first_100_bins(filepath, a=0.172, b=-5.92):
    counts = read_ortec_chn(filepath)
    channels = np.arange(len(counts))
    energy = channel_to_energy(channels, a, b)

    print(f"{'Channel':>7} {'Energy (keV)':>15} {'Counts':>10}")
    print('-' * 35)
    for ch in range(min(100, len(counts))):
        print(f"{ch:7d} {energy[ch]:15.3f} {counts[ch]:10d}")

# Example usage:
# Replace with the path to your .CHN file
filepath = "C:\\Users\\livia\\Downloads\\College\\PHYS 4410\\N-0\\Data\\4-30-25\\cs-137 (pb backing).Chn"
print_first_100_bins(filepath)

Channel    Energy (keV)     Counts
-----------------------------------
      0          -5.920          0
      1          -5.748          0
      2          -5.576          0
      3          -5.404          0
      4          -5.232          0
      5          -5.060          0
      6          -4.888          0
      7          -4.716          0
      8          -4.544          0
      9          -4.372          0
     10          -4.200          0
     11          -4.028          0
     12          -3.856          0
     13          -3.684          0
     14          -3.512          0
     15          -3.340          0
     16          -3.168          0
     17          -2.996          0
     18          -2.824          0
     19          -2.652          0
     20          -2.480          0
     21          -2.308          0
     22          -2.136          0
     23          -1.964          0
     24          -1.792          0
     25          -1.620          0
     26          -1